In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pulp import *

In [2]:
import docplex

In [3]:
import cplex

## Reading in Dataset

In [140]:
CUT_OFF = 1099


In [4]:
docplex

<module 'docplex' from '/Users/daniel/opt/anaconda3/envs/walmart2/lib/python3.8/site-packages/docplex/__init__.py'>

### Fixed Cost of each WTP

In [5]:
fixed_cost = pd.read_excel("../dataset2/fixed_cost.xlsx", index_col=0)
fixed_cost.head()

,Fixed Cost
WTP,
Ampang Intake,11638
Batang Kali,11011
Bernam River Head,11602
WTP1,4899
WTP2,11773


### Capacity of each WTP

In [6]:
capacity = pd.read_excel("../dataset2/capacity.xlsx", index_col=0)
capacity.head()

,Capacity
WTP,
Ampang Intake,2.641160e+07
Batang Kali,2.858907e+07
Bernam River Head,2.993616e+07
WTP1,2.728402e+07
WTP2,2.898516e+07


In [7]:
capacity.loc["WTP1", "Capacity"]

27284019.78365

### Distribution Loss

In [8]:
distribution_loss = pd.read_excel("../dataset2/distribution_loss.xlsx", index_col=0)
distribution_loss.head()

,Ampang Intake,Batang Kali,Bernam River Head,WTP1,WTP2,WTP3,WTP4,WTP5,WTP6,WTP7,...,WTP33,WTP34,WTP35,WTP36,WTP37,WTP38,WTP39,WTP40,Wangsa Maju,Sungai Tengi
DMZ001,0.31838,0.34819,0.23241,0.26049,0.34462,0.28837,0.30351,0.28379,0.33333,0.30272,...,0.29068,0.28830,0.29829,0.35956,0.26697,0.30082,0.27045,0.28345,0.27237,0.33081
DMZ002,0.30544,0.32145,0.29041,0.26520,0.30354,0.25078,0.31171,0.26908,0.24993,0.34083,...,0.31445,0.24484,0.35810,0.34966,0.31344,0.25027,0.29574,0.27309,0.32620,0.30501
DMZ003,0.32782,0.29775,0.30906,0.33927,0.31701,0.29337,0.26027,0.31059,0.30448,0.32909,...,0.24614,0.32459,0.23055,0.31726,0.31519,0.31940,0.29601,0.28883,0.33544,0.26115
DMZ004,0.35452,0.28941,0.25286,0.30884,0.32873,0.27913,0.26008,0.33860,0.32277,0.28385,...,0.32659,0.26638,0.28412,0.34979,0.26636,0.28850,0.28361,0.29359,0.25466,0.26176
DMZ005,0.28498,0.28067,0.29266,0.32907,0.29612,0.33973,0.24868,0.34067,0.25028,0.28086,...,0.31274,0.31326,0.29827,0.27726,0.29009,0.23659,0.27612,0.35060,0.32065,0.27252


In [9]:
distribution_loss.loc["DMZ001", "WTP1"]

0.26049

### Links between WTP and DMZ

In [10]:
linkage = pd.read_excel("../dataset2/linkage2.xlsx", index_col=0)
linkage.head()

,Ampang Intake,Batang Kali,Bernam River Head,WTP1,WTP2,WTP3,WTP4,WTP5,WTP6,WTP7,...,WTP33,WTP34,WTP35,WTP36,WTP37,WTP38,WTP39,WTP40,Wangsa Maju,Sungai Tengi
DMZ001,1,1,1,1,0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,0
DMZ002,1,1,1,1,0,1,1,1,1,1,...,1,0,1,1,1,0,1,1,0,0
DMZ003,1,1,1,1,1,1,1,1,0,1,...,1,1,0,1,1,1,1,1,0,1
DMZ004,1,1,1,1,1,1,1,1,1,1,...,1,1,0,0,1,1,1,1,0,1
DMZ005,1,1,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0,1,1,0


### Variable Cost for each combination of (DMZ, WTP)

In [11]:
variable_cost = pd.read_excel("../dataset2/variable_costs.xlsx", index_col=0)
variable_cost.head()

,Ampang Intake,Batang Kali,Bernam River Head,WTP1,WTP2,WTP3,WTP4,WTP5,WTP6,WTP7,...,WTP33,WTP34,WTP35,WTP36,WTP37,WTP38,WTP39,WTP40,Wangsa Maju,Sungai Tengi
DMZ001,9,14,9,12,24,13,21,20,8,10,...,21,23,11,9,17,7,19,18,5,22
DMZ002,12,15,23,11,18,23,17,7,7,22,...,9,9,10,23,13,18,9,15,9,11
DMZ003,5,20,19,8,9,17,7,12,18,9,...,21,21,19,15,15,5,6,16,11,21
DMZ004,22,14,15,6,12,8,9,19,17,11,...,15,9,9,24,20,8,19,13,15,5
DMZ005,22,10,7,21,22,12,21,20,9,18,...,8,14,18,23,20,14,7,5,6,5


### Transport Cost for each combination of (DMZ, WTP)

In [12]:
# transport_cost = pd.read_excel("../dataset/freight_costs.xlsx", index_col=0)
# transport_cost.head()

### Demand for each DMZ

In [141]:
demand = pd.read_excel("../dataset2/demand_chisq.xlsx", index_col=0)
demand.head()

,Demand
DMZ001,160044.62961
DMZ002,150056.74766
DMZ003,141891.56807
DMZ004,194644.00762
DMZ005,126901.55527


In [142]:
demand = demand.iloc[:CUT_OFF,:]

In [143]:
demand.loc["DMZ001", "Demand"]

160044.62961

In [144]:
# demand["Demand"] = pd.to_numeric(demand["Demand"], downcast="float")
# demand.head()

## Optimisation

In [145]:
# List of all the WTPs
wtp = list(capacity.index)

# List of all the DMZs
dmz = list(demand.index)

# List of (DMZ, WTP) pairs
dmz_wtp_pairs = [(d, w) for d in dmz for w in wtp]

In [146]:
print(wtp[:5])
print("Number of WTP:", len(wtp) )

print(dmz[:5])
print("Number of DMZ:", len(dmz) )

print("Number of DMZ-WTP Pairs:", len(dmz_wtp_pairs))

['Ampang Intake', 'Batang Kali', 'Bernam River Head', 'WTP1', 'WTP2']
Number of WTP: 45
['DMZ001', 'DMZ002', 'DMZ003', 'DMZ004', 'DMZ005']
Number of DMZ: 1099
Number of DMZ-WTP Pairs: 49455


In [147]:
demand.loc[dmz[0], "Demand"]

160044.62961

In [148]:
# Creating the Linear Optimisation Class
model = LpProblem("Optimising water supply", LpMinimize)

In [149]:
# Creating Decision Variables
output = LpVariable.dicts("Volume", dmz_wtp_pairs, lowBound=0, upBound=None, cat='continuous')

In [150]:
# fixed_cost.loc[wtp[0], "Fixed Cost"]

In [151]:
output[("DMZ001", "WTP1")]

Volume_('DMZ001',_'WTP1')

In [152]:
# Define the Objective Function

## Without transport cost
model += \
     lpSum([fixed_cost.loc[w, "Fixed Cost"] * 1000 for w in wtp]) + \
     lpSum([(variable_cost.loc[d, w]) * output[(d, w)] for d in dmz for w in wtp])


## With transport cost
# model += \
#      lpSum([fixed_cost.loc[w, "Fixed Cost"] * 1000 for w in wtp]) + \
#      lpSum([(variable_cost.loc[d, w] + transport_cost.loc[d, w]) * output[(d, w)] for d in dmz for w in wtp])

In [153]:
curr_dmz=dmz[1]
curr_wtp=wtp[0]

In [154]:
linkage.loc[curr_dmz, curr_wtp]

1

In [155]:
1- distribution_loss.loc[curr_dmz, curr_wtp]

0.6945600000000001

In [156]:
output[(curr_dmz, curr_wtp)] * (1- distribution_loss.loc[curr_dmz, curr_wtp]) * (linkage.loc[curr_dmz, curr_wtp])

0.6945600000000001*Volume_('DMZ002',_'Ampang_Intake') + 0.0

In [157]:
# type(demand.loc["DMZ1", "Demand"])
# type(1-distribution_loss.loc["DMZ1", wtp[0]])

In [158]:
capacity.loc["Ampang Intake", "Capacity"]

26411595.83813

In [159]:
distribution_loss.loc["DMZ001", "Ampang Intake"]

0.31838

In [160]:
# ## Meet demand for each DMZ
# for d in dmz:
#     for w in wtp:
#         # No linkage constraint
#         if linkage.loc[d, w] == 0 :
#             model += output[(d,w)] == 0
    
#     model += LpConstraint(
#         lpSum([ (output[(d, w)] * (1-distribution_loss.loc[d, w])) for w in wtp]),
#         sense=LpConstraintGE,
#         rhs= demand.loc[d, "Demand"],
#         name=d
#         )


In [161]:
# # Adding Constraints

## Meet demand for each DMZ
for d in dmz:
    model += lpSum([ (output[(d, w)] * (1-distribution_loss.loc[d, w]) * (linkage.loc[d, w])) for w in wtp]) >= demand.loc[d, "Demand"]

## Within the WTP capacity
for w in wtp:
    model += lpSum([output[(d, w)] for d in dmz]) <= capacity.loc[w, "Capacity"]

## No linkage constraint





# Adding Constraints


# ## Within the WTP capacity
# for w in wtp:
#     model += LpConstraint(
#         lpSum([output[(d, w)] for d in dmz]),
#         sense = LpConstraintLE,
#         rhs = capacity.loc[w, "Capacity"]*1000,
#         name = w.replace(" ", "")
#         )





In [162]:
# # Solve the model
solver = CPLEX_PY()
solver.buildSolverModel(model)

#Modify the solvermodel
# solver.solverModel.parameters.timelimit.set(1200)

#Solve P
solver.callSolver(model)
status = solver.findSolutionValues(model)

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 0 rows and 5025 columns.
Reduced MIP has 1144 rows, 44430 columns, and 88860 nonzeros.
Reduced MIP has 0 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.20 sec. (29.34 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 1144 rows, 44430 columns, and 88860 nonzeros.
Reduced MIP has 0 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.13 sec. (34.70 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.01 sec. (10.31 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0     0      integral     0   1.28831e+09   1.28831e+09     1099    0.00%
Elapsed time = 0.36 s

In [163]:
# # Solve the model
# model.solve(PULP_CBC_CMD(maxSeconds=1000, msg=1, fracGap=0))

In [80]:
# # Solve the model
# model.solve()

In [81]:
# Solve the model with cplex
model.solve(CPLEX_PY())

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 0 rows and 4540 columns.
Reduced MIP has 1044 rows, 40415 columns, and 80830 nonzeros.
Reduced MIP has 0 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.18 sec. (26.72 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 1044 rows, 40415 columns, and 80830 nonzeros.
Reduced MIP has 0 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.12 sec. (31.60 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.01 sec. (9.21 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0     0      integral     0   1.17253e+09   1.17253e+09      999    0.00%
Elapsed time = 0.34 se

1

In [164]:
print("Total Costs = {:,} ($/Month)".format(int(value(model.objective))))
print('\n' + "Status: {}".format(LpStatus[model.status]))

Total Costs = 1,677,238,466 ($/Month)

Status: Optimal


In [83]:
soln_dict = {i.name: i.varValue for i in model.variables()}

In [85]:
not_satisfied = []

for c in model.constraints.values():
    c_dict = c.toDict()
    # print(c_dict)

    satisfied = False
    
    LHS = sum([soln_dict[i['name']]*i['value'] for i in c_dict['coefficients']])
    LHS = LHS + c_dict['constant']
    
    if c_dict['sense'] == 0:
        satisfied = (LHS == 0)
   
    if c_dict['sense'] == -1:
        satisfied = (LHS <= 0)
    
    if c_dict['sense'] == 1:
        satisfied = (LHS >= 0)
   
    # print(c)
    
    if satisfied:
        #print('LHS: ', LHS)
        print('is satisfied')
    else:
        #print('LHS: ', LHS)
        print(c_dict["name"],'not satisfied', "LHS:", LHS)
        not_satisfied.append(c_dict["name"])

is satisfied
is satisfied
None not satisfied LHS: -2.9103830456733704e-11
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
None not satisfied LHS: -2.9103830456733704e-11
is satisfied
is satisfied
None not satisfied LHS: -2.9103830456733704e-11
is satisfied
is satisfied
is satisfied
None not satisfied LHS: -2.9103830456733704e-11
is satisfied
is satisfied
None not satisfied LHS: -2.9103830456733704e-11
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
None not satisfied LHS: -2.9103830456733704e-11
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfied
is satisfi

In [86]:
"DMZ001" in not_satisfied

False

In [87]:
"DMZ107" in not_satisfied

False

In [88]:
len(not_satisfied)

72

## Results

In [165]:
dict_wtp = {}
dict_dmz = {}

In [166]:
# df = pd.DataFrame(0, index=dmz, columns = wtp)
# df

df = pd.DataFrame()
df

""


In [167]:
# Getting the results
for v in model.variables():
    # print(v.name, v.varValue)
    name = v.name.replace("Volume_", "").replace("_", "")
    # print(name)

    combi = eval(name)
    # print(combi[0])

    curr_dmz = combi[0]
    curr_wtp = combi[1]
    volume = v.varValue

    # print("DMZ: ", dmz, " ", "WTP: ", wtp, " ", "Value: ", volume)

    df.loc[curr_dmz, curr_wtp] = volume


In [168]:
# Supply of water from each WTP to each DMZ (including distribution loss)
df

,AmpangIntake,BatangKali,BernamRiverHead,SungaiTengi,WTP1,WTP10,WTP11,WTP12,WTP13,WTP14,...,WTP38,WTP39,WTP4,WTP40,WTP5,WTP6,WTP7,WTP8,WTP9,WangsaMaju
DMZ001,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,219953.313648
DMZ002,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
DMZ003,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,208480.117646,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
DMZ004,0.0,0.0,0.0,263659.524843,4.210867e-11,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
DMZ005,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DMZ995,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
DMZ996,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,209710.919408
DMZ997,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,338351.287334,0.000000
DMZ998,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


In [169]:
# df.columns = ["Ampang Intake", "Batang Kali", "Bername River Head", "Sungai Tengi", "Wangsa Maju"]
# df = df[["Ampang Intake", "Batang Kali", "Bername River Head", "Wangsa Maju", "Sungai Tengi", ]]

In [170]:
# Supply of water from each WTP to each DMZ (including distribution loss)
# df

### Renaming the columns for distribution loss to match result df

In [171]:
col_list = df.columns.tolist()
print(col_list[:5])

temp_col_list = distribution_loss.columns.tolist()
dist_col_list = [w.replace(" ", "") for w in temp_col_list]

print(dist_col_list[:5])

['AmpangIntake', 'BatangKali', 'BernamRiverHead', 'SungaiTengi', 'WTP1']
['AmpangIntake', 'BatangKali', 'BernamRiverHead', 'WTP1', 'WTP2']


In [172]:
result_distribution_loss = distribution_loss.iloc[:CUT_OFF, :].copy()
result_distribution_loss.columns = dist_col_list

In [173]:
result_distribution_loss = result_distribution_loss[col_list]

### Accounting for distribution loss

In [174]:
useful_amount = 1- result_distribution_loss.values

In [175]:
temp = df.values * useful_amount
temp

array([[     0.        ,      0.        ,      0.        , ...,
             0.        ,      0.        , 160044.62961   ],
       [     0.        ,      0.        ,      0.        , ...,
             0.        ,      0.        ,      0.        ],
       [     0.        ,      0.        ,      0.        , ...,
             0.        ,      0.        ,      0.        ],
       ...,
       [     0.        ,      0.        ,      0.        , ...,
             0.        , 218124.92440566,      0.        ],
       [     0.        ,      0.        ,      0.        , ...,
             0.        ,      0.        ,      0.        ],
       [     0.        ,      0.        ,      0.        , ...,
             0.        ,      0.        ,      0.        ]])

In [176]:
temp.shape

(1099, 45)

In [177]:
# Supply of water from each WTP to each DMZ, after accounting for water loss
final_df = pd.DataFrame(temp, columns = wtp, index=dmz)
final_df.round(3)

,Ampang Intake,Batang Kali,Bernam River Head,WTP1,WTP2,WTP3,WTP4,WTP5,WTP6,WTP7,...,WTP33,WTP34,WTP35,WTP36,WTP37,WTP38,WTP39,WTP40,Wangsa Maju,Sungai Tengi
DMZ001,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,160044.630
DMZ002,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000
DMZ003,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,141891.568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000
DMZ004,0.0,0.0,0.0,194644.008,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000
DMZ005,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DMZ1095,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000
DMZ1096,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,151321.108
DMZ1097,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,218124.924,0.000
DMZ1098,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000


In [178]:
# wtp

In [179]:
supply = final_df.sum(axis=1)
supply[:5]

DMZ001    160044.62961
DMZ002    150056.74766
DMZ003    141891.56807
DMZ004    194644.00762
DMZ005    126901.55527
dtype: float64

In [180]:
sum(final_df.loc["DMZ001",:])

160044.62961

In [181]:
demand.loc["DMZ001", "Demand"]

160044.62961

In [182]:
prob = []
print("       ", "Supply", "            ", "Demand")
for curr_dmz in dmz:
    a = supply[curr_dmz]
    b = demand.loc[curr_dmz, "Demand"]
    if not (np.isclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)) and a <= b:
        print(curr_dmz, supply[curr_dmz], ">=", demand.loc[curr_dmz, "Demand"], "?")
        prob.append(curr_dmz)

        Supply              Demand
DMZ102 172582.47090789533 >= 219335.13647 ?
DMZ103 160445.5197442441 >= 162679.28167 ?
DMZ105 168450.95968492227 >= 192934.71108 ?
DMZ107 136591.24868740886 >= 203053.81513 ?
DMZ110 149929.28502039905 >= 167461.64841 ?
DMZ113 140725.61994925325 >= 170376.29337 ?
DMZ114 149467.3390395652 >= 200656.84275 ?
DMZ115 127167.6372572889 >= 211408.31361 ?
DMZ117 163592.7897013613 >= 176386.39911 ?
DMZ118 135291.9599854408 >= 148959.6222 ?
DMZ119 170655.2034475564 >= 184917.60021 ?
DMZ120 173602.94249961755 >= 174459.2079 ?
DMZ121 138163.50938456427 >= 233416.07347 ?
DMZ122 214317.70585017596 >= 214621.31332 ?
DMZ123 158390.17854867634 >= 206664.34339 ?
DMZ124 128367.51668296549 >= 190825.24332 ?
DMZ125 140871.0227035897 >= 168214.77578 ?
DMZ126 170264.20263808165 >= 220918.23369 ?
DMZ127 150509.2960158221 >= 155538.47874 ?
DMZ130 141110.71925552154 >= 201130.21082 ?
DMZ134 130740.16188245974 >= 174478.69247 ?
DMZ135 139266.3230368626 >= 174213.98184 ?
DMZ138 1

In [107]:
# prob = []
# for curr_dmz in dmz:
#     # print(supply[curr_dmz], ">=", demand.loc[curr_dmz, "Demand"], "?")
#     # print(supply[curr_dmz] >= demand.loc[curr_dmz, "Demand"])
#     a = supply[curr_dmz]
#     b = demand.loc[curr_dmz, "Demand"]
#     if not (np.isclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)) and a <= b:
#         print(curr_dmz, supply[curr_dmz], ">=", demand.loc[curr_dmz, "Demand"], "?")
#         prob.append(curr_dmz)
#     # elif (np.isclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)):
#     #     print(True)
#     # else: 
#     #     print(supply[curr_dmz], ">=", demand.loc[curr_dmz, "Demand"], "?")
#     #     print(supply[curr_dmz] >= demand.loc[curr_dmz, "Demand"])

In [54]:
len(prob)

0

In [55]:
check_capacity = final_df.sum(axis=0)

In [56]:
prob_wtp = []
for curr_wtp in wtp:
    # print(curr_dmz, check_capacity[curr_wtp], "<=", capacity.loc[curr_wtp, "Capacity"], "?")
    # print(check_capacity[curr_wtp] <= capacity.loc[curr_wtp, "Capacity"])
    a = check_capacity[curr_wtp]
    b = capacity.loc[curr_wtp, "Capacity"]
    if not (np.isclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)) and a >= b:
        print(curr_dmz, check_capacity[curr_wtp], "<=", capacity.loc[curr_wtp, "Capacity"], "?")
        prob_wtp.append(curr_wtp)
    # elif (np.isclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)):
    #     print(True)
    # else: 
    #     # print(curr_dmz, check_capacity[curr_wtp], "<=", capacity.loc[curr_wtp, "Capacity"], "?")
    #     print(check_capacity[curr_wtp] <= capacity.loc[curr_wtp, "Capacity"])

In [234]:
len(prob_wtp)

0

In [236]:
df.head()

,AmpangIntake,BatangKali,BernamRiverHead,SungaiTengi,WTP1,WTP10,WTP11,WTP12,WTP13,WTP14,...,WTP38,WTP39,WTP4,WTP40,WTP5,WTP6,WTP7,WTP8,WTP9,WangsaMaju
DMZ001,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,219953.31
DMZ002,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
DMZ003,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,208480.12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
DMZ004,0.0,0.0,0.0,263659.52,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
DMZ005,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00


In [237]:
final_df.head()

,Ampang Intake,Batang Kali,Bernam River Head,WTP1,WTP2,WTP3,WTP4,WTP5,WTP6,WTP7,...,WTP33,WTP34,WTP35,WTP36,WTP37,WTP38,WTP39,WTP40,Wangsa Maju,Sungai Tengi
DMZ001,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,160044.626955
DMZ002,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
DMZ003,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,141891.569672,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
DMZ004,0.0,0.0,0.0,194644.004045,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
DMZ005,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [238]:
# Export results

df.to_excel("../dataset2/result_without_loss.xlsx", index=True)
final_df.to_excel("../dataset2/result.xlsx", index=True)

In [132]:
demand.loc["DMZ005", "Demand"]

126901.55527